# Linear Programming: Multi Commodity Production Planning

By: Mansur M. Arief

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/analytics-project-simt-its/analytics-project-simt-its.github.io/blob/main/notebooks/pyomo_multi-commodity_production_planning.ipynb)

In [ ]:
%pip install pyomo
%pip install gurobipy

In [33]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd


# Model

Objective: find the product grades to maximize profits


Set:
- $I$: set of grade types
- $J$: set of raw materials
- $K$: set of processing methods
- $T$: set of time periods


Parameter:
- $p_i$: price of grade $i$
- $r^{bulk, max}_{i}$: max rate of bulk material for grade $i$
- $r^{bulk, min}_{i}$: min rate of bulk material for grade $i$
- $r^{granule, max}_{i}$: max rate of granule material for grade $i$
- $r^{granule, min}_{i}$: min rate of granule material for grade $i$
- $d_{it}$: demand of grade $i$ at time $t$
- $m_{ik}$: 1 if grade $i$ can be produced by processing method $k$, 0 otherwise
- $n_{ijk}$: quantity of raw material $j$ needed to produce 1 ton of grade $i$ by processing method $k$
- $c_j$: cost of raw material $j$
- $h_j$: holding cost of raw material $j$
- $s_i$: holding cost for grade $i$
- $l_j$: lead time of raw material $j$
- $b$: bagging cost/ton (in USD)
- $t_{ik}$: time to produce 1 ton of grade $i$ by processing method $k$
- $e_{ik}$: energy consumption to produce 1 ton of grade $i$ by processing method $k$
- $u^{min}, u^{max}$: min and max material to order 



Variable:
- $x_{ikt}$: quantity of grade $i$ produced by processing method $k$ at time $t$
- $y_{it}$: quantity of grade $i$ in stock at time $t$
- $z_{it}$: quantity of grade $i$ sold at time $t$
- $u_{jt}$: quantity of raw material $j$ ordered at time $t$
- $v_{jt}$: quantity of raw material $j$ in stock at time $t$




Objective function:
- Revenue: $\sum_{i \in I} \sum_{t \in T} p_i z_{it}$
- Cost of energy: $\sum_{i \in I} \sum_{k \in K} \sum_{t \in T} e_{ik} x_{ikt}$
- Cost of manpower: $\sum_{i \in I} \sum_{k \in K} \sum_{t \in T} t_{ik} x_{ikt}$
- Cost of bagging: $\sum_{i \in I} \sum_{k \in K} \sum_{t \in T} b x_{ikt}$
- Cost of holding products inventory: $\sum_{i \in I} \sum_{t \in T} s_i y_{it}$
- Cost of raw materials: $\sum_{j \in J} \sum_{t \in T} c_j u_{jt}$
- Cost of holding raw materials inventory: $\sum_{j \in J} \sum_{t \in T} h_j v_{jt}$


$$\begin{aligned}
\text{maximize}_{\mathbf x, \mathbf y, \mathbf z, \mathbf u, \mathbf v} 
& \quad \bigg( \sum_{i \in I} \sum_{t \in T} p_i z_{it} \bigg) - \bigg( \sum_{i \in I} \sum_{k \in K} \sum_{t \in T} (e_{ik} - t_{ik} - b )x_{ikt} \bigg) - \bigg( \sum_{i \in I} \sum_{t \in T}  s_i y_{it} \bigg) - \bigg( \sum_{j \in J} \sum_{t \in T} (c_j  u_{jt} + h_j v_{jt} ) \bigg)  \\
\end{aligned}$$



In [674]:
# Create a dummy model
n_grades = 3 #i
n_materials = 5 #j
n_methods = 2 #k
n_periods = 10 #t

# set seed and parameter
np.random.seed(2024)
p_i = np.random.randint(3, 8, n_grades)*400
d_it = np.random.randint(20,30, (n_grades, n_periods))
m_ik = np.random.randint(1, 10, (n_grades, n_methods)) > 5
n_ijk = np.random.uniform(1, 4, (n_grades, n_materials, n_methods))
c_j = np.random.randint(100, 200, n_materials)
h_j = np.random.uniform(0.5, 1.0, n_materials)
s_i = np.random.uniform(0.5, 1.5, (n_grades))

b = np.random.rand()
t_ik = np.random.uniform(0.1, 0.2, (n_grades, n_methods))
e_ik = np.random.uniform(0.1, 0.5, (n_grades, n_methods))
u_min = 0
u_max = 9999
y_i1 = [30 for i in range(n_grades)]
v_j1 = [100 for j in range(n_materials)]


In [675]:
#Instantiate the model
model = pyo.AbstractModel()

# Define the sets
model.I = pyo.RangeSet(n_grades)
model.J = pyo.RangeSet(n_materials)
model.K = pyo.RangeSet(n_methods)
model.T = pyo.RangeSet(n_periods)
model.Tplus1 = pyo.RangeSet(2, n_periods)

# Define the parameters
model.p = pyo.Param(model.I, initialize=lambda model, i: p_i[i-1])
model.d = pyo.Param(model.I, model.T, initialize=lambda model, i, t: d_it[i-1, t-1])
model.m = pyo.Param(model.I, model.K, initialize=lambda model, i, k: m_ik[i-1, k-1])
model.n = pyo.Param(model.I, model.J, model.K, initialize=lambda model, i, j, k: n_ijk[i-1, j-1, k-1])
model.c = pyo.Param(model.J, initialize=lambda model, j: c_j[j-1])
model.h = pyo.Param(model.J, initialize=lambda model, j: h_j[j-1])
model.s = pyo.Param(model.I, initialize=lambda model, i: s_i[i-1])
model.b = pyo.Param(initialize=b)
model.t = pyo.Param(model.I, model.K, initialize=lambda model, i, k: t_ik[i-1, k-1])
model.e = pyo.Param(model.I, model.K, initialize=lambda model, i, k: e_ik[i-1, k-1])
model.u_min = pyo.Param(initialize=u_min)
model.u_max = pyo.Param(initialize=u_max)
model.y_i1 = pyo.Param(model.I, initialize=lambda model, i: y_i1[i-1])
model.v_j1 = pyo.Param(model.J, initialize=lambda model, j: v_j1[j-1])

# Define the variables
model.x = pyo.Var(model.I, model.K, model.T, within=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)
model.z = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)
model.u = pyo.Var(model.J, model.T, within=pyo.NonNegativeReals)
model.v = pyo.Var(model.J, model.T, within=pyo.NonNegativeReals)

In [676]:
# Define the objective function
def revenue(model):
    return sum(model.p[i]*model.z[i,t] for i in model.I for t in model.T)

def cost_energy(model):
    return sum(model.e[i,k]*model.x[i,k,t] for i in model.I for k in model.K for t in model.T)

def cost_manpower(model):
    return sum(model.t[i,k]*model.x[i,k,t] for i in model.I for k in model.K for t in model.T)

def cost_bagging(model):
    return sum(model.b*model.x[i,k,t] for i in model.I for k in model.K for t in model.T)

def cost_inventory_product(model):
    return sum(model.s[i]* model.y[i,t] for i in model.I for t in model.T)

def cost_material(model):
    return sum(model.c[j]*model.u[j,t] for j in model.J for t in model.T)

def cost_inventory_material(model):
    return sum(model.h[j]*model.v[j,t] for j in model.J for t in model.T)

def objective_rule(model):
    return revenue(model) - cost_energy(model) - cost_manpower(model) - cost_bagging(model) - cost_inventory_product(model) - cost_material(model) - cost_inventory_material(model)
model.Objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

Constraint

- All demand must be satisfied: $z_{it} = d_{it} \quad \forall i \in I, t \in T$
- Product inventory balance: $y_{it} = y_{i(t-1)} + \sum_{k \in K} x_{ikt} - z_{it} \quad \forall i \in I, t \in T$
- Sold product does not exceed what is available: $z_{it} \leq y_{it} + \sum_{k \in K} x_{ikt} \quad \forall i \in I, t \in T$


In [677]:
#define constraint
def demand_satisfied(model, i, t):
    return model.z[i,t]  ==   model.d[i,t]
model.demand_satisfied = pyo.Constraint(model.I, model.T, rule=demand_satisfied)

def product_inventory_balance(model, i, t):
    return model.y[i,t] == model.y[i,t-1] + sum(model.x[i,k,t-1] for k in model.K) - model.z[i,t-1]
model.product_inventory_balance = pyo.Constraint(model.I, model.Tplus1, rule=product_inventory_balance)

def sales_not_exceed_available(model, i, t):
    return model.z[i,t] <= model.y[i,t] + sum(model.x[i,k,t] for k in model.K)
model.sales_not_exceed_available = pyo.Constraint(model.I, model.T, rule=sales_not_exceed_available)


- Raw material inventory balance: $v_{jt} = v_{j(t-1)} + u_{j(t-1)} - \sum_{k \in K} \sum_{i \in I} n_{ijk} x_{ikt} \quad \forall j \in J, t \in T$

- Production does not exceed available material: $x_{ikt} \leq n_{ijk} v_{jt} \quad \forall i \in I, j \in J, k \in K, t \in T$

- Min order quantity: $u_{jt} \geq u^{min} \quad \forall j \in J, t \in T$


In [678]:
def material_inventory_balance(model, j, t):
    return model.v[j,t] == model.v[j,t-1] + model.u[j, t-1] - sum(model.n[i,j,k]*model.x[i,k,t-1] for i in model.I for k in model.K)
model.material_inventory_balance = pyo.Constraint(model.J, model.Tplus1, rule=material_inventory_balance)

def material_availability(model, j, t):
    return sum(model.n[i,j,k]*model.x[i,k,t-1] for i in model.I for k in model.K) <= model.v[j,t]
model.material_availability = pyo.Constraint(model.J,model.Tplus1, rule=material_availability)

def min_order_quantity(model, j, t):
    return model.u[j,t] >= model.u_min
model.min_order_quantity = pyo.Constraint(model.J, model.T, rule=min_order_quantity)


- Initial product inventory: $y_{i1} = y_{i, init} \quad \forall i \in I$
- Initial material inventory: $v_{j1} = v_{j, init} \quad \forall j \in J$


In [679]:

def initial_inventory_product(model, i):
    return model.y[i,1] == model.y_i1[i]
model.initial_inventory_product = pyo.Constraint(model.I, rule=initial_inventory_product)

def initial_inventory_material(model, j):
    return model.v[j,1] == model.v_j1[j]
model.initial_inventory_material = pyo.Constraint(model.J, rule=initial_inventory_material)

In [680]:
instance = model.create_instance()
instance.pprint()

16 Set Declarations
    d_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*T :   30 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10)}
    demand_satisfied_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*T :   30 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10)}
    e_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*K :    6 : {(1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2)}
    m_index : Size=1, Index=N

In [681]:

solver = pyo.SolverFactory('gurobi') #change the path to the gurobi.sh file in your system
result = solver.solve(instance, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-23
Read LP format model from file /var/folders/d5/r2v0z0z17nnfzysx1xgwf9jc0000gn/T/tmp3qlqc51o.pyomo.lp
Reading time = 0.00 seconds
x221: 236 rows, 221 columns, 1064 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.4.0 23E224)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 236 rows, 221 columns and 1064 nonzeros
Model fingerprint: 0x6dc32df5
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e-01, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 125 rows and 80 columns
Presolve time: 0.00s
Presolved: 111 rows, 141 columns, 885 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9614871e+32   1.578653e+32   1.961487e+02      0s
     208    2.3481372e+05   0.

In [686]:
#print the inventory of product 
print("\nInitial inventory (row: grade, column: period)")
print("===================================")
y = pd.DataFrame()
for i in instance.I:
    for t in instance.T:
        y.loc[i, t] = round(instance.y[i,t].value, 2)
print(y)


#print the production plan as a table
for k in instance.K:
    x = pd.DataFrame()
    for i in instance.I:
        for t in instance.T:
            x.loc[i, t] = round(instance.x[i,k,t].value, 2)
    print("\nProduct produced by method", k)
    print("===================================")
    print(x)

#print the amount of product sold as a table
print("\nProduct sold")
print("===================================")
z = pd.DataFrame()
for i in instance.I:
    for t in instance.T:
        z.loc[i, t] = round(instance.z[i,t].value, 2)
print(z)



Initial inventory (row: grade, column: period)
     1       2       3       4       5      6      7      8      9    10
1  30.0  100.68  122.02  117.68  100.02  84.69  64.52  43.94  24.65  0.0
2  30.0  102.69  125.54  124.47  114.93  94.16  68.27  43.83  20.61  0.0
3  30.0  105.20  128.80  129.10  118.75  99.07  78.23  50.81  25.60  0.0

Product produced by method 1
       1      2      3      4     5     6     7     8     9    10
1    0.00   0.00   0.00   0.00  0.00  0.00  0.00  0.00  0.00  0.0
2   99.69  49.85  24.92  12.46  6.23  3.12  1.56  0.78  0.39  0.0
3  101.20  50.60  25.30  12.65  6.32  3.16  1.58  0.79  0.40  0.0

Product produced by method 2
      1      2      3      4     5     6     7     8     9     10
1  90.68  45.34  22.67  11.33  5.67  2.83  1.42  0.71  0.35  21.0
2   0.00   0.00   0.00   0.00  0.00  0.00  0.00  0.00  0.00  26.0
3   0.00   0.00   0.00   0.00  0.00  0.00  0.00  0.00  0.00  24.0

Product sold
     1     2     3     4     5     6     7     8     9    

In [683]:
#print the material order as a table
print("\nMaterial order")
print("===================================")
u = pd.DataFrame()
for j in instance.J:
    for t in instance.T:
        u.loc[j, t] = round(instance.u[j,t].value, 2)
print(u)


#print the material inventory as a table
print("\nMaterial inventory")
print("===================================")
v = pd.DataFrame()
for j in instance.J:
    for t in instance.T:
        v.loc[j, t] = round(instance.v[j,t].value, 2)
        
print(v)


Material order
        1    2    3    4    5    6    7    8    9    10
1  1398.59  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1323.72  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  1744.32  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4   858.43  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5   738.59  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

Material inventory
      1       2       3       4       5      6      7      8     9     10
1  100.0  749.30  374.65  187.32   93.66  46.83  23.42  11.71  5.85  2.93
2  100.0  711.86  355.93  177.96   88.98  44.49  22.25  11.12  5.56  2.78
3  100.0  922.16  461.08  230.54  115.27  57.63  28.82  14.41  7.20  3.60
4  100.0  479.22  239.61  119.80   59.90  29.95  14.98   7.49  3.74  1.87
5  100.0  419.30  209.65  104.82   52.41  26.21  13.10   6.55  3.28  1.64


In [684]:
#print objective value
print("\nObjective value")
print("===================================")
print(round(pyo.value(instance.Objective), 2))


Objective value
234813.72
